# DeepSDF

<font color='red'>NOTE : </font> <font color='green'>Lets use the notebooks only for testing and debugging purposes. We need a .py file that we can use in the notebook. </font>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

# TODO: Fill this according to your drive loc
# DAVID: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '/DeepSDF/'
# STANLEY: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/'
# DEEPAK:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ROB 498 3D Robot Perception/3D-RP-Project/DeepSDF/' # Edit and replace | Don't delete my directory lol
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'DeepSDF/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['data', 'requirements.txt', 'utils', 'params.json', 'reconstruction.py', 'hello_testing.py', '__pycache__', 'PCN.ipynb', 'Reconstruct.ipynb', 'Grasp Generation.ipynb', 'dataset.py', '.ipynb_checkpoints', 'config_files', 'model', 'results', 'scripts', 'obj_paths.pkl', 'Dataset.ipynb', 'eval']


If you did everything right, you should see something like this:
```
['DeepSDF.ipynb', 'data', 'models', 'requirements.txt', 'utils', 'params.json', 'Reconstruct.ipynb', 'Dataset.ipynb', 'hello_testing.py', 'model.py', '__pycache__']
```

<font color='red'>NOTE:</font> You don't have to clone the shared drive.
1. Create a shortcut of the shared folder (DeepSDF) and place in your drive.
2. Then change the directory on the notebook according to your gdrive.

In [ ]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

from hello_testing import hello
hello()

model_path = os.path.join(GOOGLE_DRIVE_PATH, 'model/model.py')
model_edit_time = time.ctime(os.path.getmtime(model_path))
print('model.py last edited on %s' % model_edit_time)

Working!
model.py last edited on Sun Nov  5 17:43:40 2023


Navigate to :

```drive > .... > 3D-RP-Project > model.py```

Double-click on the file to open it on the side.

## model.py

## train.py

In [ ]:
import torch
import model.model_sdf as sdf_model
import torch.optim as optim
import data.dataset_sdf as dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import results.runs_sdf as runs
from utils.utils_deepsdf import SDFLoss_multishape
import os
from datetime import datetime
import numpy as np
import time
from utils import utils_deepsdf
import results
# from torch.utils.tensorboard import SummaryWriter
import yaml
import config_files

In [ ]:
# Select device. The 'mps' device (macOS M1 architecture) is not supported as it cannot currently handle weith normalisation.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cpu


In [ ]:
class Trainer():
    def __init__(self, train_cfg):
        self.train_cfg = train_cfg

    def __call__(self):
        # directories
        self.timestamp_run = datetime.now().strftime('%d_%m_%H%M%S')   # timestamp to use for logging data
        self.runs_dir = os.path.dirname(runs.__file__)               # directory fo all runs
        self.run_dir = os.path.join(self.runs_dir, self.timestamp_run)  # directory for this run
        if not os.path.exists(self.run_dir):
            os.makedirs(self.run_dir)

        # Logging
        # self.writer = SummaryWriter(log_dir=self.run_dir)
        # self.log_path = os.path.join(self.run_dir, 'settings.yaml')
        # with open(self.log_path, 'w') as f:
        #     yaml.dump(self.train_cfg, f)

        # calculate num objects in samples_dictionary, wich is the number of keys
        samples_dict_path = os.path.join(os.path.dirname(results.__file__), f'samples_dict_{train_cfg["dataset"]}.npy')
        samples_dict = np.load(samples_dict_path, allow_pickle=True).item()

        # instantiate model and optimisers
        self.model = sdf_model.SDFModel(
                self.train_cfg['num_layers'],
                self.train_cfg['skip_connections'],
                inner_dim=self.train_cfg['inner_dim'],
                latent_size=self.train_cfg['latent_size']
            ).float().to(device)

        # define optimisers
        self.optimizer_model = optim.Adam(self.model.parameters(), lr=self.train_cfg['lr_model'], weight_decay=0)

        # generate a unique random latent code for each shape
        self.latent_codes = utils_deepsdf.generate_latent_codes(self.train_cfg['latent_size'], samples_dict)
        self.optimizer_latent = optim.Adam([self.latent_codes], lr=self.train_cfg['lr_latent'], weight_decay=0)

        # Load pretrained weights and optimisers to continue training
        if self.train_cfg['pretrained']:
            # load pretrained weights
            self.model.load_state_dict(torch.load(self.train_cfg['pretrain_weights'], map_location=device))

            # load pretrained optimisers
            self.optimizer_model.load_state_dict(torch.load(self.train_cfg['pretrain_optim_model'], map_location=device))

            # retrieve latent codes from results.npy file
            results_path = self.train_cfg['pretrain_optim_model'].split(os.sep)
            results_path[-1] = 'results.npy'
            results_path = os.sep.join(results_path)
            # load latent codes from results.npy file
            results_latent_codes = np.load(results_path, allow_pickle=True).item()
            self.latent_codes = torch.tensor(results_latent_codes['best_latent_codes']).float().to(device)
            self.optimizer_latent = optim.Adam([self.latent_codes], lr=self.train_cfg['lr_latent'], weight_decay=0)
            self.optimizer_latent.load_state_dict(torch.load(self.train_cfg['pretrain_optim_latent'], map_location=device))

        if self.train_cfg['lr_scheduler']:
            self.scheduler_model =  torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer_model, mode='min', factor=self.train_cfg['lr_multiplier'], patience=self.train_cfg['patience'], threshold=0.0001, threshold_mode='rel')
            self.scheduler_latent =  torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer_latent, mode='min', factor=self.train_cfg['lr_multiplier'], patience=self.train_cfg['patience'], threshold=0.0001, threshold_mode='rel')

        # get data
        train_loader, val_loader = self.get_loaders()
        self.results = {
            'best_latent_codes' : []
        }

        best_loss = 10000000000
        start = time.time()
        for epoch in range(self.train_cfg['epochs']):
            print(f'============================ Epoch {epoch} ============================')
            self.epoch = epoch

            avg_train_loss = self.train(train_loader)

            with torch.no_grad():
                avg_val_loss = self.validate(val_loader)

                if avg_val_loss < best_loss:
                    best_loss = np.copy(avg_val_loss)
                    best_weights = self.model.state_dict()
                    best_latent_codes = self.latent_codes.detach().cpu().numpy()
                    optimizer_model_state = self.optimizer_model.state_dict()
                    optimizer_latent_state = self.optimizer_latent.state_dict()

                    np.save(os.path.join(self.run_dir, 'results.npy'), self.results)
                    torch.save(best_weights, os.path.join(self.run_dir, 'weights.pt'))
                    torch.save(optimizer_model_state, os.path.join(self.run_dir, 'optimizer_model_state.pt'))
                    torch.save(optimizer_latent_state, os.path.join(self.run_dir, 'optimizer_latent_state.pt'))
                    self.results['best_latent_codes'] = best_latent_codes

                if self.train_cfg['lr_scheduler']:
                    self.scheduler_model.step(avg_val_loss)
                    self.scheduler_latent.step(avg_val_loss)

                    # self.writer.add_scalar('Learning rate (model)', self.scheduler_model._last_lr[0], epoch)
                    # self.writer.add_scalar('Learning rate (latent)', self.scheduler_latent._last_lr[0], epoch)

        end = time.time()
        print(f'Time elapsed: {end - start} s')

    def get_loaders(self):
        data = dataset.SDFDataset(self.train_cfg['dataset'])

        if self.train_cfg['clamp']:
            data.data['sdf'] = torch.clamp(data.data['sdf'], -self.train_cfg['clamp_value'], self.train_cfg['clamp_value'])

        train_size = int(0.85 * len(data))
        val_size = len(data) - train_size
        train_data, val_data = random_split(data, [train_size, val_size])
        train_loader = DataLoader(
                train_data,
                batch_size=self.train_cfg['batch_size'],
                shuffle=True,
                drop_last=True
            )
        val_loader = DataLoader(
            val_data,
            batch_size=self.train_cfg['batch_size'],
            shuffle=False,
            drop_last=True
            )
        return train_loader, val_loader

    def generate_xy(self, batch):
        """
        Combine latent code and coordinates.
        Return:
            - x: latent codes + coordinates, torch tensor shape (batch_size, latent_size + 3)
            - y: ground truth sdf, shape (batch_size, 1)
            - latent_codes_indices_batch: all latent class indices per sample, shape (batch size, 1).
                                            e.g. [[2], [2], [1], ..] eaning the batch contains the 2nd, 2nd, 1st latent code
            - latent_batch_codes: all latent codes per sample, shape (batch_size, latent_size)
        Return ground truth as y, and the latent codes for this batch.
        """
        latent_classes_batch = batch[0][:, 0].view(-1, 1).to(torch.long)               # shape (batch_size, 1)
        coords = batch[0][:, 1:]                                  # shape (batch_size, 3)
        latent_codes_batch = self.latent_codes[latent_classes_batch.view(-1)]    # shape (batch_size, 128)

        x = torch.hstack((latent_codes_batch, coords))                  # shape (batch_size, 131)
        y = batch[1]     # (batch_size, 1)

        return x, y, latent_classes_batch.view(-1), latent_codes_batch

    def train(self, train_loader):
        total_loss = 0.0
        iterations = 0.0
        self.model.train()
        for batch in train_loader:
            # batch[0]: [class, x, y, z], shape: (batch_size, 4)
            # batch[1]: [sdf], shape: (batch size)
            iterations += 1.0

            self.optimizer_model.zero_grad()
            self.optimizer_latent.zero_grad()

            x, y, latent_codes_indices_batch, latent_codes_batch = self.generate_xy(batch)

            predictions = self.model(x)  # (batch_size, 1)
            if self.train_cfg['clamp']:
                predictions = torch.clamp(predictions, -self.train_cfg['clamp_value'], self.train_cfg['clamp_value'])

            loss_value, loss_rec, loss_latent = self.train_cfg['loss_multiplier'] * SDFLoss_multishape(y, predictions, x[:, :self.train_cfg['latent_size']], sigma=self.train_cfg['sigma_regulariser'])
            loss_value.backward()

            self.optimizer_latent.step()
            self.optimizer_model.step()
            total_loss += loss_value.data.cpu().numpy()

        avg_train_loss = total_loss/iterations
        print(f'Training: loss {avg_train_loss}')
        # self.writer.add_scalar('Training loss', avg_train_loss, self.epoch)

        return avg_train_loss

    def validate(self, val_loader):
        total_loss = 0.0
        total_loss_rec = 0.0
        total_loss_latent = 0.0
        iterations = 0.0
        self.model.eval()

        for batch in val_loader:
            # batch[0]: [class, x, y, z], shape: (batch_size, 4)
            # batch[1]: [sdf], shape: (batch size)
            iterations += 1.0

            x, y, _, latent_codes_batch = self.generate_xy(batch)

            predictions = self.model(x)  # (batch_size, 1)
            if train_cfg['clamp']:
                predictions = torch.clamp(predictions, -train_cfg['clamp_value'], train_cfg['clamp_value'])

            loss_value, loss_rec, loss_latent = self.train_cfg['loss_multiplier'] * SDFLoss_multishape(y, predictions, latent_codes_batch, self.train_cfg['sigma_regulariser'])
            total_loss += loss_value.data.cpu().numpy()
            total_loss_rec += loss_rec.data.cpu().numpy()
            total_loss_latent += loss_latent.data.cpu().numpy()

        avg_val_loss = total_loss/iterations
        avg_loss_rec = total_loss_rec/iterations
        avg_loss_latent = total_loss_latent/iterations
        print(f'Validation: loss {avg_val_loss}')
        # self.writer.add_scalar('Validation loss', avg_val_loss, self.epoch)
        # self.writer.add_scalar('Reconstruction loss', avg_loss_rec, self.epoch)
        # self.writer.add_scalar('Latent code loss', avg_loss_latent, self.epoch)

        return avg_val_loss

train_cfg_path = os.path.join(os.path.dirname(config_files.__file__), 'train_sdf.yaml')
with open(train_cfg_path, 'rb') as f:
    train_cfg = yaml.load(f, Loader=yaml.FullLoader)

trainer = Trainer(train_cfg)
trainer()

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


============================ Epoch 0 ============================
Training: loss 0.008144985840282422
Validation: loss 0.00463022875375656
============================ Epoch 1 ============================
Training: loss 0.004043672948955018
Validation: loss 0.00354206330733923
============================ Epoch 2 ============================
Training: loss 0.003352849525988628
Validation: loss 0.0032060814994636483
============================ Epoch 3 ============================
Training: loss 0.0030024126717534847
Validation: loss 0.0028085731648109765
============================ Epoch 4 ============================
Training: loss 0.0027640594756703466
Validation: loss 0.002659822670188513
============================ Epoch 5 ============================
Training: loss 0.002515872583313351
Validation: loss 0.0022458664118971583
============================ Epoch 6 ============================
Training: loss 0.0022134549893555442
Validation: loss 0.0021291230124584767
===============

KeyboardInterrupt: ignored